## Install & Import Statements
Import: See the inline comments for a brief rational of each library.

Notes: The SPARQLWrapper libary provides tools for querying SPARQL endpoints. The sparql_dataframe library can be used with SPARQLWrapper to convert JSON results from a SPARQL query directly to a Pandas dataframe. The mapclassify library is required by GeoPandas for its .explore functionality.

In [2]:
from branca.element import Figure                                  # For controlling the size of the final map
import folium                                                      # For map layer control
import geopandas as gpd                                            # For geospatial dataframes
import pandas as pd                                                # For dataframes
from shapely import wkt                                            # For working with WKT coordinates in a GeoDataFrame
from SPARQLWrapper import SPARQLWrapper, JSON, GET, POST, DIGEST   # For querying SPARQL endpoints
import sparql_dataframe                                            # For converting SPARQL query results to Pandas dataframes
from rdflib import Graph
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from itertools import cycle

## Variable Initialization
A SPARQLWrapper is created to access the SAWGraph repository for the SAWGraph project.

In [3]:
%%capture
pd.options.display.width = 240
endpointPOST = 'http://Shirlys-MacBook-Pro.local:7200/repositories/streamnet'
sparqlPOST = SPARQLWrapper(endpointPOST)

## Queries

These queries directly access data in Salmon Knowledge Graph that houses the GBIF, Streamnet results. They use federation to access additional data in KWG repositories.


The query is executed and returned as a dataframe.

In [4]:
query1="""
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX fish-ont: <http://purl.dataone.org/fish-ont/>
PREFIX prov-o: <https://www.w3.org/TR/prov-o/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX streamnet-ont: <https://knb.ecoinformatics.org/knb/streamnet/lod/ontology/>
PREFIX nosa-ont: <https://knb.ecoinformatics.org/knb/streamnet/nosa/lod/ontology/>
PREFIX nosar: <https://knb.ecoinformatics.org/knb/streamnet/nosa/lod/resource/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?recovery_domain ?water_body ?wkt ?sampled_date
WHERE {
    ?nosa_obs_coll a nosa-ont:Escapement_ObservationCollection;
    		fish-ont:hasTemporalContext ?sampled_date;
    		sosa:hasFeatureOfInterest ?location.
    ?location geo:hasGeometry ?geo;
    		fish-ont:locatedIn ?water_body_iri;
    		fish-ont:associatedRegion ?recovery_domain_iri.
	?geo geo:hasGeometry|geo:asWKT ?wkt.
    ?water_body_iri rdfs:label ?water_body.
    ?recovery_domain_iri rdfs:label ?recovery_domain.
}
"""
sparqlPOST.setQuery(query1)
sparqlPOST.setReturnFormat(JSON)

try:
    # results = sparqlPOST.query().convert()
    # for result in results["results"]["bindings"]:
    #     print(result)
    nosa_df1 = sparql_dataframe.get(endpointPOST, query1)
    # print(df)
except Exception as e:
    print(f"An error occurred: {e}")

In [5]:
query2="""
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX fish-ont: <http://purl.dataone.org/fish-ont/>
PREFIX prov-o: <https://www.w3.org/TR/prov-o/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX streamnet-ont: <https://knb.ecoinformatics.org/knb/streamnet/lod/ontology/>
PREFIX nosa-ont: <https://knb.ecoinformatics.org/knb/streamnet/nosa/lod/ontology/>
PREFIX nosar: <https://knb.ecoinformatics.org/knb/streamnet/nosa/lod/resource/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?recovery_domain ?water_body ?wkt ?salmon_species ?nosaij
WHERE {
    ?nosa_obs_coll a nosa-ont:Escapement_ObservationCollection;
    		fish-ont:hasTemporalContext ?time;
    		sosa:hasFeatureOfInterest ?location;
    		fish-ont:ofSpeciesType ?salmon_species;
    		sosa:hasMember ?nosa_obs.
    ?nosa_obs sosa:observedProperty nosa-ont:salmonCharacteristic.NOSAIJ;
            sosa:hasSimpleResult ?nosaij.
    ?location geo:hasGeometry ?geo;
    		fish-ont:locatedIn ?water_body_iri;
    		fish-ont:associatedRegion ?recovery_domain_iri.
	?geo geo:hasGeometry|geo:asWKT ?wkt.
    ?water_body_iri rdfs:label ?water_body.
    ?recovery_domain_iri rdfs:label ?recovery_domain.
    FILTER (?time = "2018"^^xsd:gYear)
}
"""
sparqlPOST.setQuery(query2)
sparqlPOST.setReturnFormat(JSON)

try:
    # results = sparqlPOST.query().convert()
    # for result in results["results"]["bindings"]:
    #     print(result)
    nosa_df2 = sparql_dataframe.get(endpointPOST, query2)
    # print(df)
except Exception as e:
    print(f"An error occurred: {e}")


query3="""
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX fish-ont: <http://purl.dataone.org/fish-ont/>
PREFIX prov-o: <https://www.w3.org/TR/prov-o/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX streamnet-ont: <https://knb.ecoinformatics.org/knb/streamnet/lod/ontology/>
PREFIX nosa-ont: <https://knb.ecoinformatics.org/knb/streamnet/nosa/lod/ontology/>
PREFIX nosar: <https://knb.ecoinformatics.org/knb/streamnet/nosa/lod/resource/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?recovery_domain ?water_body ?wkt ?salmon_species ?nosaij
WHERE {
    ?nosa_obs_coll a nosa-ont:Escapement_ObservationCollection;
    		fish-ont:hasTemporalContext ?time;
    		sosa:hasFeatureOfInterest ?location;
    		fish-ont:ofSpeciesType ?salmon_species;
    		sosa:hasMember ?nosa_obs.
    ?nosa_obs sosa:observedProperty nosa-ont:salmonCharacteristic.NOSAIJ;
            sosa:hasSimpleResult ?nosaij.
    ?location geo:hasGeometry ?geo;
    		fish-ont:locatedIn ?water_body_iri;
    		fish-ont:associatedRegion ?recovery_domain_iri.
	?geo geo:hasGeometry|geo:asWKT ?wkt.
    ?water_body_iri rdfs:label ?water_body.
    ?recovery_domain_iri rdfs:label ?recovery_domain.
    FILTER (?time = "2019"^^xsd:gYear)
}
"""
sparqlPOST.setQuery(query3)
sparqlPOST.setReturnFormat(JSON)

try:
    # results = sparqlPOST.query().convert()
    # for result in results["results"]["bindings"]:
    #     print(result)
    nosa_df3 = sparql_dataframe.get(endpointPOST, query3)
    # print(df)
except Exception as e:
    print(f"An error occurred: {e}")

query4="""
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX fish-ont: <http://purl.dataone.org/fish-ont/>
PREFIX prov-o: <https://www.w3.org/TR/prov-o/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX streamnet-ont: <https://knb.ecoinformatics.org/knb/streamnet/lod/ontology/>
PREFIX nosa-ont: <https://knb.ecoinformatics.org/knb/streamnet/nosa/lod/ontology/>
PREFIX nosar: <https://knb.ecoinformatics.org/knb/streamnet/nosa/lod/resource/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?recovery_domain ?water_body ?wkt ?salmon_species ?nosaij
WHERE {
    ?nosa_obs_coll a nosa-ont:Escapement_ObservationCollection;
    		fish-ont:hasTemporalContext ?time;
    		sosa:hasFeatureOfInterest ?location;
    		fish-ont:ofSpeciesType ?salmon_species;
    		sosa:hasMember ?nosa_obs.
    ?nosa_obs sosa:observedProperty nosa-ont:salmonCharacteristic.NOSAIJ;
            sosa:hasSimpleResult ?nosaij.
    ?location geo:hasGeometry ?geo;
    		fish-ont:locatedIn ?water_body_iri;
    		fish-ont:associatedRegion ?recovery_domain_iri.
	?geo geo:hasGeometry|geo:asWKT ?wkt.
    ?water_body_iri rdfs:label ?water_body.
    ?recovery_domain_iri rdfs:label ?recovery_domain.
    FILTER (?time = "2020"^^xsd:gYear)
}
"""
sparqlPOST.setQuery(query4)
sparqlPOST.setReturnFormat(JSON)

try:
    # results = sparqlPOST.query().convert()
    # for result in results["results"]["bindings"]:
    #     print(result)
    nosa_df4 = sparql_dataframe.get(endpointPOST, query4)
    # print(df)
except Exception as e:
    print(f"An error occurred: {e}")

query5="""
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX fish-ont: <http://purl.dataone.org/fish-ont/>
PREFIX prov-o: <https://www.w3.org/TR/prov-o/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX streamnet-ont: <https://knb.ecoinformatics.org/knb/streamnet/lod/ontology/>
PREFIX nosa-ont: <https://knb.ecoinformatics.org/knb/streamnet/nosa/lod/ontology/>
PREFIX nosar: <https://knb.ecoinformatics.org/knb/streamnet/nosa/lod/resource/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?recovery_domain ?water_body ?wkt ?salmon_species ?nosaij
WHERE {
    ?nosa_obs_coll a nosa-ont:Escapement_ObservationCollection;
    		fish-ont:hasTemporalContext ?time;
    		sosa:hasFeatureOfInterest ?location;
    		fish-ont:ofSpeciesType ?salmon_species;
    		sosa:hasMember ?nosa_obs.
    ?nosa_obs sosa:observedProperty nosa-ont:salmonCharacteristic.NOSAIJ;
            sosa:hasSimpleResult ?nosaij.
    ?location geo:hasGeometry ?geo;
    		fish-ont:locatedIn ?water_body_iri;
    		fish-ont:associatedRegion ?recovery_domain_iri.
	?geo geo:hasGeometry|geo:asWKT ?wkt.
    ?water_body_iri rdfs:label ?water_body.
    ?recovery_domain_iri rdfs:label ?recovery_domain.
    FILTER (?time = "2021"^^xsd:gYear)
}
"""
sparqlPOST.setQuery(query3)
sparqlPOST.setReturnFormat(JSON)

try:
    # results = sparqlPOST.query().convert()
    # for result in results["results"]["bindings"]:
    #     print(result)
    nosa_df5 = sparql_dataframe.get(endpointPOST, query5)
    # print(df)
except Exception as e:
    print(f"An error occurred: {e}")

## Visualizing on a map

**bold text** Create GeoPandas dataframes for mapping

Convert the above dataframes to GeoDataFrames, setting the WKT columns as the geometry columns and setting the CRS to WGS 84.

In [6]:
nosa_df1['wkt'] = nosa_df1['wkt'].apply(wkt.loads)
nosa_df1= nosa_df1[['recovery_domain', 'water_body', 'wkt', 'sampled_date']]

nosa_df2['wkt'] = nosa_df2['wkt'].apply(wkt.loads)
nosa_df2= nosa_df2[['recovery_domain', 'water_body', 'wkt', 'salmon_species', 'nosaij']]

nosa_df3['wkt'] = nosa_df3['wkt'].apply(wkt.loads)
nosa_df3= nosa_df3[['recovery_domain', 'water_body', 'wkt', 'salmon_species', 'nosaij']]

nosa_df4['wkt'] = nosa_df4['wkt'].apply(wkt.loads)
nosa_df4= nosa_df4[['recovery_domain', 'water_body', 'wkt', 'salmon_species', 'nosaij']]

nosa_df5['wkt'] = nosa_df5['wkt'].apply(wkt.loads)
nosa_df5= nosa_df5[['recovery_domain', 'water_body', 'wkt', 'salmon_species', 'nosaij']]

In [7]:
%%capture
nosa_df1 = gpd.GeoDataFrame(nosa_df1, geometry='wkt')
nosa_df1.set_crs(epsg=4326, inplace=True, allow_override=True)

nosa_df2 = gpd.GeoDataFrame(nosa_df2, geometry='wkt')
nosa_df2.set_crs(epsg=4326, inplace=True, allow_override=True)

nosa_df3 = gpd.GeoDataFrame(nosa_df3, geometry='wkt')
nosa_df3.set_crs(epsg=4326, inplace=True, allow_override=True)

nosa_df4 = gpd.GeoDataFrame(nosa_df4, geometry='wkt')
nosa_df4.set_crs(epsg=4326, inplace=True, allow_override=True)

nosa_df5 = gpd.GeoDataFrame(nosa_df5, geometry='wkt')
nosa_df5.set_crs(epsg=4326, inplace=True, allow_override=True)

# Visualizing on a map

In [8]:
from folium.plugins import MarkerCluster
from folium import CircleMarker

In [9]:
#%%capture
fac_color = 'black'
boundweight = 5

map = nosa_df1.explore(
        style_kwds=dict(weight=boundweight, style_function=lambda x: {"color": "Green"}), 
                     tooltip=True,
                     name='NOSA Escapement Locations',
                     show=True)
# m = folium.Map(location=[gbif_abundance_gdf.geometry.centroid.y.mean(), 
#                          gbif_abundance_gdf.geometry.centroid.x.mean()], 
#                zoom_start=10)

# for _, row in gbif_abundance_gdf.iterrows():
#     lat, lon = row.wkt.y, row.wkt.x
#     folium.CircleMarker(
#         location=[lat, lon],
#         radius=5,  # Size of the circle
#         color="blue",  # Circle border color
#         fill=True,
#         fill_color="cyan",  # Fill color
#         fill_opacity=0.6,
#         popup=f"Value: {row['value']}, Year: {row['year']}",
#         tooltip=row['obs_point'],
#     ).add_to(m)

## Show map

The map is created inside a Figure box to control its size. This displays the samples sized by the max measured concentration of any PFAs chemical

In [10]:
# map.save('Salmon observation points in Bresle River)

fig = Figure(width=1200, height=800)
fig.add_child(map)

# m

## Create map with multiple layers

Each GeoDataFrame is a layer in the final map.

In [11]:
from folium import LayerControl
fac_color = 'black'
boundweight = 5

map = nosa_df2.explore(
        style_kwds=dict(weight=boundweight, style_function=lambda x: {"color": "Green"}),
                     tooltip=True,
                     name='NOSA Escapement Locations - 2019',
                     show=True)
nosa_df3.explore(m=map,
                  color='blue',
                  style_kwds=dict(weight=boundweight),
                  tooltip=True,
                  name='NOSA Escapement Locations - 2020',
                  show=True)
nosa_df4.explore(m=map,
                color='Orange',
                style_kwds=dict(weight=boundweight),
                tooltip=True,
                name='NOSA Escapement Locations - 2021',
                show=True)
nosa_df4.explore(m=map,
                color='DeepPink',
                style_kwds=dict(weight=boundweight),
                tooltip=True,
                name='NOSA Escapement Locations - 2022',
                show=True)
# m = folium.Map(location=[gbif_abundance_gdf.geometry.centroid.y.mean(), 
#                          gbif_abundance_gdf.geometry.centroid.x.mean()], 
#                zoom_start=10)

# for _, row in gbif_abundance_gdf.iterrows():
#     lat, lon = row.wkt.y, row.wkt.x
#     folium.CircleMarker(
#         location=[lat, lon],
#         radius=5,  # Size of the circle
#         color="blue",  # Circle border color
#         fill=True,
#         fill_color="cyan",  # Fill color
#         fill_opacity=0.6,
#         popup=f"Value: {row['value']}, Year: {row['year']}",
#         tooltip=row['obs_point'],
#     ).add_to(m)
LayerControl().add_to(map)
map

## Retrieve Controlled Vocabulary 

Get terms or classification schemes across all Streamnet datasets. These terms form a common integrating layer over all datasets, i.e., NOSA, RperS, JuvOut, 

Retrieve all Salmon species sampled/observed in various Streamnet datasets

In [ ]:
query_species ="""
PREFIX : <http://ecoinformatics.org/oboe/oboe.1.2/oboe.owl#>
PREFIX oboe-core: <http://ecoinformatics.org/oboe/oboe.1.2/oboe-core.owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX fish-ont: <http://purl.dataone.org/fish-ont/>
PREFIX prov-o: <https://www.w3.org/TR/prov-o/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX streamnet-ont: <https://knb.ecoinformatics.org/knb/streamnet/lod/ontology/>
PREFIX nosa-ont: <https://knb.ecoinformatics.org/knb/streamnet/nosa/lod/ontology/>
PREFIX nosar: <https://knb.ecoinformatics.org/knb/streamnet/nosa/lod/resource/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX juvout-ont: <https://knb.ecoinformatics.org/knb/streamnet/JUVOUT/lod/ontology/>
PREFIX sar-ont: <https://knb.ecoinformatics.org/knb/streamnet/SAR/lod/ontology/>
PREFIX rpers-ont: <https://knb.ecoinformatics.org/knb/streamnet/rpers/lod/ontology/>
SELECT DISTINCT ?dataset_iri ?dataset_name ?salmon_species_iri ?salmon_species_name
WHERE {
    ?obs_coll a ?o;
    	fish-ont:ofSpeciesType ?salmon_species_iri;
    	prov-o:wasDerivedFrom ?dataset_iri.
    ?dataset_iri oboe-core:title ?dataset_name.
    ?salmon_species_iri rdfs:label ?salmon_species_name.
    FILTER (?o = juvout-ont:Escapement_ObservationCollection || ?o = nosa-ont:Escapement_ObservationCollection || ?o = sar-ont:Escapement_ObservationCollection || ?o = rpers-ont:Escapement_ObservationCollection)
}

"""
sparqlPOST.setQuery(query_species)
sparqlPOST.setReturnFormat(JSON)

try:
    # results = sparqlPOST.query().convert()
    # for result in results["results"]["bindings"]:
    #     print(result)
    species_df = sparql_dataframe.get(endpointPOST, query_species)
    print(species_df)
except Exception as e:
    print(f"An error occurred: {e}")

                                          dataset_iri                             dataset_name                           salmon_species_iri salmon_species_name
0   http://ecoinformatics.org/oboe/oboe.1.2/oboe-c...            Juvenile Outmigrants (JuvOut)  http://purl.dataone.org/odo/SALMON_00000570     Steelhead trout
1   http://ecoinformatics.org/oboe/oboe.1.2/oboe-c...            Juvenile Outmigrants (JuvOut)  http://purl.dataone.org/odo/SALMON_00000513      Chinook salmon
2   http://ecoinformatics.org/oboe/oboe.1.2/oboe-c...            Juvenile Outmigrants (JuvOut)  http://purl.dataone.org/odo/SALMON_00000514         Chum salmon
3   http://ecoinformatics.org/oboe/oboe.1.2/oboe-c...            Juvenile Outmigrants (JuvOut)  http://purl.dataone.org/odo/SALMON_00000517      Sockeye salmon
4   http://ecoinformatics.org/oboe/oboe.1.2/oboe-c...            Juvenile Outmigrants (JuvOut)  http://purl.dataone.org/odo/SALMON_00000515         Coho salmon
5   http://ecoinformatics.org/oboe/oboe.

Retrieve all measured characteristics in various Streamnet datasets.

In [20]:
query_characteristic ="""
PREFIX : <http://ecoinformatics.org/oboe/oboe.1.2/oboe.owl#>
PREFIX oboe-core: <http://ecoinformatics.org/oboe/oboe.1.2/oboe-core.owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX fish-ont: <http://purl.dataone.org/fish-ont/>
PREFIX prov-o: <https://www.w3.org/TR/prov-o/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX streamnet-ont: <https://knb.ecoinformatics.org/knb/streamnet/lod/ontology/>
PREFIX nosa-ont: <https://knb.ecoinformatics.org/knb/streamnet/nosa/lod/ontology/>
PREFIX nosar: <https://knb.ecoinformatics.org/knb/streamnet/nosa/lod/resource/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX juvout-ont: <https://knb.ecoinformatics.org/knb/streamnet/JUVOUT/lod/ontology/>
PREFIX sar-ont: <https://knb.ecoinformatics.org/knb/streamnet/SAR/lod/ontology/>
PREFIX rpers-ont: <https://knb.ecoinformatics.org/knb/streamnet/rpers/lod/ontology/>
SELECT DISTINCT ?dataset_name ?measured_characteristic_name ?measured_characteristic_description
WHERE {
    ?obs_coll a ?o;
    	sosa:hasMember ?obs;
    	prov-o:wasDerivedFrom ?dataset_iri.
    ?dataset_iri oboe-core:title ?dataset_name.
    ?obs sosa:observedProperty ?measured_characteristic_iri.
    ?measured_characteristic_iri rdfs:label ?measured_characteristic_name;
    	rdfs:comment ?measured_characteristic_description.
    FILTER (?o = juvout-ont:Escapement_ObservationCollection || ?o = nosa-ont:Escapement_ObservationCollection || ?o = sar-ont:Escapement_ObservationCollection || ?o = rpers-ont:Escapement_ObservationCollection)
}

"""
sparqlPOST.setQuery(query_characteristic)
sparqlPOST.setReturnFormat(JSON)

try:
    # results = sparqlPOST.query().convert()
    # for result in results["results"]["bindings"]:
    #     print(result)
    characteristic_df = sparql_dataframe.get(endpointPOST, query_characteristic)
    print(characteristic_df)
    characteristic_df.to_html("output_characteristic.html")
except Exception as e:
    print(f"An error occurred: {e}")

                               dataset_name measured_characteristic_name                measured_characteristic_description
0   Natural Origin Spawner Abundance (NOSA)                     Age3Prop  The proportion of natural origin fish that wer...
1   Natural Origin Spawner Abundance (NOSA)                     Age4Prop  The proportion of natural origin fish that wer...
2   Natural Origin Spawner Abundance (NOSA)                     Age5Prop  The proportion of natural origin fish that wer...
3   Natural Origin Spawner Abundance (NOSA)                     Age6Prop  The proportion of natural origin fish that wer...
4   Natural Origin Spawner Abundance (NOSA)                       NOSAIJ  Including jacks, point estimate for NOSA or na...
5   Natural Origin Spawner Abundance (NOSA)                       pHOSij  Point estimate for the proportion of fish spaw...
6   Natural Origin Spawner Abundance (NOSA)                        TSAIJ          Total spawner abundance, including jacks.
7   Natu

## Cross-Graph Integration 
Establishing data connection across graphs (maintained by other groups/organizations)

In [ ]:
from IPython.display import display
query_kwg="""
PREFIX kwg-ont: <http://stko-kwg.geog.ucsb.edu/lod/ontology/>
PREFIX : <http://ecoinformatics.org/oboe/oboe.1.2/oboe.owl#>
PREFIX oboe-core: <http://ecoinformatics.org/oboe/oboe.1.2/oboe-core.owl#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX time: <http://www.w3.org/2006/time#>
PREFIX fish-ont: <http://purl.dataone.org/fish-ont/>
PREFIX prov-o: <https://www.w3.org/TR/prov-o/>
PREFIX geo: <http://www.opengis.net/ont/geosparql#>
PREFIX sosa: <http://www.w3.org/ns/sosa/>
PREFIX streamnet-ont: <https://knb.ecoinformatics.org/knb/streamnet/lod/ontology/>
PREFIX nosa-ont: <https://knb.ecoinformatics.org/knb/streamnet/nosa/lod/ontology/>
PREFIX nosar: <https://knb.ecoinformatics.org/knb/streamnet/nosa/lod/resource/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX juvout-ont: <https://knb.ecoinformatics.org/knb/streamnet/JUVOUT/lod/ontology/>
PREFIX sar-ont: <https://knb.ecoinformatics.org/knb/streamnet/SAR/lod/ontology/>
PREFIX rpers-ont: <https://knb.ecoinformatics.org/knb/streamnet/rpers/lod/ontology/>
SELECT ?location ?recovery_domain ?water_body ?region
WHERE {
    ?nosa_obs_coll a nosa-ont:Escapement_ObservationCollection;
    		fish-ont:hasTemporalContext ?time;
    		sosa:hasFeatureOfInterest ?location;
    		sosa:hasMember ?nosa_obs.
    ?nosa_obs sosa:observedProperty nosa-ont:salmonCharacteristic.NOSAIJ;
            sosa:hasSimpleResult ?nosaij.
    ?location kwg-ont:sfWithin ?region.
    ?location geo:hasGeometry ?geo;
    		fish-ont:locatedIn ?water_body_iri;
    		fish-ont:associatedRegion ?recovery_domain_iri.
#	?geo geo:hasGeometry|geo:asWKT ?wkt.
    ?water_body_iri rdfs:label ?water_body.
    ?recovery_domain_iri rdfs:label ?recovery_domain.
    FILTER (?time = "2018"^^xsd:gYear)
}

"""
sparqlPOST.setQuery(query_kwg)
sparqlPOST.setReturnFormat(JSON)

try:
    # results = sparqlPOST.query().convert()
    # for result in results["results"]["bindings"]:
    #     print(result)
    nosa_df10 = sparql_dataframe.get(endpointPOST, query_kwg)
    print(nosa_df10)
    nosa_df10.to_html("output_kwg-integrated.html")
except Exception as e:
    print(f"An error occurred: {e}")

                                             location            recovery_domain                                         water_body                                             region
0   https://knb.ecoinformatics.org/knb/streamnet/l...          Interior Columbia                                       Alpowa Creek  http://stko-kwg.geog.ucsb.edu/lod/resource/s2....
1   https://knb.ecoinformatics.org/knb/streamnet/l...          Interior Columbia                                       Alpowa Creek  http://stko-kwg.geog.ucsb.edu/lod/resource/adm...
2   https://knb.ecoinformatics.org/knb/streamnet/l...          Interior Columbia                                      Antoine Creek  http://stko-kwg.geog.ucsb.edu/lod/resource/s2....
3   https://knb.ecoinformatics.org/knb/streamnet/l...          Interior Columbia                                      Antoine Creek  http://stko-kwg.geog.ucsb.edu/lod/resource/adm...
4   https://knb.ecoinformatics.org/knb/streamnet/l...          Interior Columbia     